# Getting Started

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
import tensorflow as tf

In [ ]:
data = pd.read_csv('../input/av-healthcare-analytics-ii/healthcare/train_data.csv')

In [ ]:
data

# Preprocessing

## Missing Values

In [ ]:
data.isnull().sum()

In [ ]:
def impute_missing_values(data, columns):
    for column in columns:
        data[column] = data[column].fillna(data[column].mean())

In [ ]:
impute_columns = ['Bed Grade', 'City_Code_Patient']

impute_missing_values(data, impute_columns)

In [ ]:
data.isnull().sum()

## Encoding

In [ ]:
data.dtypes

In [ ]:
def get_categorical_uniques(data):
    categorical_columns = [column for column in data.dtypes.index if data.dtypes[column] == 'object']
    categorical_uniques = {column: data[column].unique() for column in categorical_columns}
    
    return categorical_uniques

In [ ]:
get_categorical_uniques(data)

In [ ]:
pd.get_dummies(data['Department'])

In [ ]:
def onehot_encode(data, columns):
    for column in columns:
        dummies = pd.get_dummies(data[column])
        data = pd.concat([data, dummies], axis=1)
        data.drop(column, axis=1, inplace=True)
    return data

In [ ]:
onehot_columns = ['Hospital_type_code', 'Hospital_region_code', 'Department', 'Ward_Type', 'Ward_Facility_Code']

In [ ]:
data = onehot_encode(data, onehot_columns)

In [ ]:
data

In [ ]:
categorical_uniques = get_categorical_uniques(data)
get_categorical_uniques(data)

In [ ]:
for column in categorical_uniques:
    categorical_uniques[column] = sorted(categorical_uniques[column])

In [ ]:
categorical_uniques

In [ ]:
unique_list = categorical_uniques['Type of Admission']
unique_list.insert(0, unique_list.pop(unique_list.index('Urgent')))
unique_list.insert(0, unique_list.pop(unique_list.index('Trauma')))

unique_list = categorical_uniques['Severity of Illness']
unique_list.insert(0, unique_list.pop(unique_list.index('Moderate')))
unique_list.insert(0, unique_list.pop(unique_list.index('Minor')))

In [ ]:
categorical_uniques

In [ ]:
stay_mappings = {value: index for index, value in enumerate(categorical_uniques['Stay'])}
stay_mappings

In [ ]:
def ordinal_encode(data, uniques):
    for column in uniques:
        data[column] = data[column].apply(lambda x: uniques[column].index(x))

In [ ]:
data['Stay']

In [ ]:
ordinal_encode(data, categorical_uniques)
data['Stay']

In [ ]:
(data.dtypes == 'object').sum()

## Scaling

In [ ]:
data

In [ ]:
data.set_index('case_id', inplace=True)

In [ ]:
y = data['Stay']
X = data.drop('Stay', axis=1)

In [ ]:
scaler = StandardScaler()

X = pd.DataFrame(scaler.fit_transform(X), index=X.index, columns=X.columns)

In [ ]:
X

# Training

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)

In [ ]:
log_model = LogisticRegression()
log_model.fit(X_train, y_train)

In [ ]:
inputs = tf.keras.Input(shape=(38,))
x = tf.keras.layers.Dense(16, activation='relu')(inputs)
x = tf.keras.layers.Dense(16, activation='relu')(x)
outputs = tf.keras.layers.Dense(11, activation='softmax')(x)

nn_model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
nn_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
batch_size = 32
epochs = 10

history = nn_model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    batch_size=batch_size,
    epochs=epochs,
)

In [ ]:
print(f"Logistic Regression Acc: {log_model.score(X_test, y_test)}")
print(f"     Neural Netowrk Acc: {nn_model.evaluate(X_test, y_test, verbose=0)[1]}")

In [ ]:
plt.figure(figsize=(14, 10))

plt.plot(range(epochs), history.history['loss'], label="Training Loss")
plt.plot(range(epochs), history.history['val_loss'], label="Validation Loss")

plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend(loc='upper right')

plt.show()

In [ ]:
np.argmin(history.history['val_loss']) + 1